In [1]:
import hydra
import torch
from tqdm.notebook import tqdm
from lib.utils.config import load_config
from lib.optimizer.framework import NeuralOptimizer
from lib.data.loader import load_intrinsics
from lib.data.loader import load_intrinsics
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.tracker.timer import TimeTracker
from collections import defaultdict
import pandas as pd


def path_to_abblation(path):
    return "_".join(path.split("/")[-3].split("_")[1:])


def eval_iterations(optimizer, datamodule, N: int = 3):
    optimizer.max_iters = N
    optimizer.max_optims = 1
    time_tracker = TimeTracker()
    # initial evaluation no optimization
    p_loss = []
    v_loss = []
    g_loss = []
    for batch in datamodule.val_dataloader():
        with torch.no_grad():
            batch = optimizer.transfer_batch_to_device(batch, "cuda", 0)
            time_tracker.start("optimize")
            out = optimizer(batch)
            time_tracker.stop("optimize")
            loss_info = optimizer.compute_loss(batch=batch, out=out)
            p_loss.append(loss_info["loss_param"])
            v_loss.append(loss_info["loss_vertices"])
            g_loss.append(loss_info["loss_geometric"])
    iters_p_loss = torch.stack(p_loss).mean().item()
    iters_g_loss = torch.stack(g_loss).mean().item()
    iters_v_loss = torch.stack(v_loss).mean().item()
    t_perf = [torch.tensor(t.time_ms) for t in list(time_tracker.tracks.values())[0]]
    iters_time = torch.stack(t_perf).median().item()
    return iters_p_loss, iters_g_loss, iters_v_loss, iters_time


def load_flame_renderer():
    # instanciate similar to training
    cfg = load_config("train", ["data=kinect"])
    K = load_intrinsics(data_dir=cfg.data.intrinsics_dir, return_tensor="pt")
    camera = Camera(
        K=K,
        width=cfg.data.width,
        height=cfg.data.height,
        near=cfg.data.near,
        far=cfg.data.far,
        scale=cfg.data.scale,
    )
    rasterizer = Rasterizer(width=camera.width, height=camera.height)
    renderer = Renderer(rasterizer=rasterizer, camera=camera)
    flame = hydra.utils.instantiate(cfg.model)
    return flame, renderer


def load_neural_optimizer(flame, renderer, path):
    cfg = load_config("train", ["data=kinect"])
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    regularize = hydra.utils.instantiate(cfg.regularize)
    neural_optimizer = NeuralOptimizer.load_from_checkpoint(
        path,
        renderer=renderer,
        flame=flame,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        weighting=weighting,
    )
    return neural_optimizer


def load_icp_optimizer(flame, renderer, overrides):
    o = ["data=kinect", "optimizer.output_dir=none"] + overrides
    cfg = load_config("train", o)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    optimizer = hydra.utils.instantiate(cfg.optimizer)
    regularize = hydra.utils.instantiate(cfg.regularize)
    icp_optimizer = hydra.utils.instantiate(
        cfg.framework,
        flame=flame,
        logger=None,
        renderer=renderer,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        optimizer=optimizer,
        weighting=weighting,
    )
    return icp_optimizer.to("cuda")


# setup the datamodule
def load_datamodule(renderer, start_frame, end_frame, jump_size):
    cfg = load_config("train", ["data=kinect"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=start_frame,
            end_frame=end_frame,
            jump_size=jump_size,
        ),
    )
    datamodule.setup("fit")
    return datamodule

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
N = 3
start_frame = 100
end_frame = 108
step_size = 0.3

# checkpoints
ours = "/home/borth/GuidedResearch/logs/2024-10-09/09-58-52_train_kinect/checkpoints/epoch_059.ckpt"
ours_syn = "/home/borth/GuidedResearch/logs/2024-10-09/09-58-52_train_syn/checkpoints/epoch_1059.ckpt"

# loadings
times = defaultdict(dict)
p_losses = defaultdict(dict)
g_losses = defaultdict(dict)
v_losses = defaultdict(dict)
flame, renderer = load_flame_renderer()

for jump_size in [1, 2, 4, 8]:
    datamodule = load_datamodule(renderer, start_frame, end_frame, jump_size)

    optimizer = load_neural_optimizer(flame, renderer, ours_syn)
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "ours-syn"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_neural_optimizer(flame, renderer, ours)
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "ours"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, [])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=1)
    key = "base"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=point2plane", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-geo"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=face2face", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-geo+reg"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=face2face", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=5)
    key = "icp-5"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

Creating GL context for cuda device 0
Successfully initialized EGL version 1.5
Successfully initialized OpenGL version 4.6.0 NVIDIA 535.183.01


In [4]:
desired_order = ["base", "icp-geo", "icp-geo+reg", "icp-5", "ours", "ours-syn"]

# Create the DataFrame for p_losses
p_losses_df = pd.DataFrame(p_losses).transpose()
p_losses_df.columns = [f"0->{c}" for c in p_losses_df.columns]
p_losses_df = p_losses_df.reindex(desired_order)
p_losses_df.columns = pd.MultiIndex.from_tuples([("FLAME (norm)", c) for c in p_losses_df.columns])

# Create the DataFrame for g_losses
g_losses_df = pd.DataFrame(g_losses).transpose()
g_losses_df.columns = [f"0->{c}" for c in g_losses_df.columns]
g_losses_df = g_losses_df.reindex(desired_order)
g_losses_df.columns = pd.MultiIndex.from_tuples([("P2P (mm)", c) for c in g_losses_df.columns])

# Create the DataFrame for v_losses
v_losses_df = pd.DataFrame(v_losses).transpose()
v_losses_df.columns = [f"0->{c}" for c in v_losses_df.columns]
v_losses_df = v_losses_df.reindex(desired_order)
v_losses_df.columns = pd.MultiIndex.from_tuples([("Vertices (mm)", c) for c in v_losses_df.columns])

# Create the DataFrame for v_losses
# time_df = pd.DataFrame(times).transpose()
# time_df.columns = [f"0->{c}" for c in time_df.columns]
# time_df = time_df.reindex(desired_order)
# time_df.columns = pd.MultiIndex.from_tuples([("Time (ms)", c) for c in time_df.columns])

time_df = pd.DataFrame(times).transpose()
time_df = pd.DataFrame(time_df.mean(axis=1))
time_df = time_df.reindex(desired_order)
time_df.columns = pd.MultiIndex.from_tuples([("Time (ms)", "")])

# Concatenate the two DataFrames
pd.concat([p_losses_df, g_losses_df, v_losses_df, time_df], axis=1)

FLAME (norm)                                P2P (mm)            \
                    0->1      0->2      0->4      0->8      0->1      0->2   
base            0.696968  1.071554  1.689559  1.377419  0.850747  1.040949   
icp-geo         0.726097  1.014169  1.381035  1.202713  0.694245  0.795113   
icp-geo+reg     0.656642  0.892561  1.140841  1.046699  0.714899  0.819876   
icp-5           0.671536  0.819405  0.984589  0.925081  0.667871  0.717319   
ours            0.578797  0.891519  1.257133  1.140351  0.698383  0.803857   
ours-syn        0.656329  0.943237  1.301150  1.180536  0.699579  0.796870   

                                Vertices (mm)                                \
                 0->4      0->8          0->1      0->2      0->4      0->8   
base         1.301811  1.276427      1.717836  2.982004  4.502889  3.711512   
icp-geo      0.900423  0.891832      1.202562  1.856837  2.749907  2.378053   
icp-geo+reg  0.929678  0.909071      1.190414  1.852666  2.552354  2.205449   
icp-5        0.768003  0.742022      1.056243  1.397354  1.802786  1.614482   
ours         0.913482  0.898444      1.042357  1.825572  2.642303  2.336560   
ours-syn     0.910913  0.903561      1.093967  1.772385  2.525091  2.277721   

             Time (ms)  
                        
base         23.262560  
icp-geo      51.155806  
icp-geo+reg  53.896903  
icp-5        89.449287  
ours         69.919943  
ours-syn     70.087015